In [1]:
import os
import tensorflow as tf
import numpy as np 
import pandas as pd
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model, model_from_json, load_model, Sequential
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import BatchNormalization,MaxPooling2D, Input, Concatenate, Dense, Dropout, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image
from pathlib import Path
import cv2 as cv
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from keras_preprocessing.image import load_img
from keras import callbacks

In [2]:
print(tf.__version__)


2.6.0


In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3809715622323802958
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2956456756
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10678836705668570121
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [4]:
physical_devices= tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))
print(tf.test.is_built_with_cuda())

Num GPUs Available:  1
True


In [5]:
train_dir = 'E:/DataSets/affect_net_sample_archive/train_class'
validation_dir ='E:/DataSets/affect_net_sample_archive/val_class'

In [6]:
#See image shape with matplotlib.pylot
img = plt.imread(os.path.join(train_dir,'class001\image0000002.jpg'))
img.shape
#print(img)
#print(img/255)

(177, 177, 3)

In [7]:
train_datagen_aug = ImageDataGenerator(rescale = 1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

valid_datagen_aug = ImageDataGenerator(rescale = 1./255)

In [8]:
train_generator_aug = train_datagen_aug.flow_from_directory(train_dir,
                                                    target_size=(177,177),
                                                    class_mode='categorical',
                                                    batch_size=32)

valid_generator_aug = train_datagen_aug.flow_from_directory(validation_dir,
                                                    target_size=(177,177),
                                                    class_mode='categorical',
                                                    batch_size=32)

Found 37553 images belonging to 8 classes.
Found 4000 images belonging to 8 classes.


In [9]:
conv_base_VGG_aug = VGG19(
    include_top=False,
    weights="imagenet",
 input_shape=(177,177,3),
    classes=8 
)

In [10]:
conv_base_VGG_aug.trainable = False

In [11]:
model_VGG_aug = models.Sequential()
model_VGG_aug.add(conv_base_VGG_aug)
model_VGG_aug.add(Flatten())
model_VGG_aug.add(Dense(128, activation='relu')) 
model_VGG_aug.add(Dropout(0.2))
model_VGG_aug.add(Dense(8, activation='softmax'))#7 categories of facial emotions

In [12]:
model_VGG_aug.compile(optimizer='adam',#sgd
             loss=tf.losses.categorical_crossentropy,
             metrics=['acc'])

In [13]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",
                                        mode ="min", 
                                        patience = 10)#the number of epochs to stop after we see no improvments

In [14]:
history_VGG_aug = model_VGG_aug.fit(train_generator_aug,
                    epochs=100,
                    verbose=1,
                    validation_data=valid_generator_aug,
                   callbacks =[earlystopping])

Epoch 1/100
1174/1174 [==============================] - 1543s 1s/step - loss: 2.0237 - acc: 0.1956 - val_loss: 1.9743 - val_acc: 0.2260
Epoch 2/100
1174/1174 [==============================] - 730s 621ms/step - loss: 1.9795 - acc: 0.2139 - val_loss: 1.9658 - val_acc: 0.2145
Epoch 3/100
1174/1174 [==============================] - 748s 637ms/step - loss: 1.9617 - acc: 0.2252 - val_loss: 1.9278 - val_acc: 0.2450
Epoch 4/100
1174/1174 [==============================] - 728s 620ms/step - loss: 1.9503 - acc: 0.2324 - val_loss: 1.9338 - val_acc: 0.2303
Epoch 5/100
1174/1174 [==============================] - 732s 623ms/step - loss: 1.9482 - acc: 0.2349 - val_loss: 1.9220 - val_acc: 0.2450
Epoch 6/100
1174/1174 [==============================] - 746s 635ms/step - loss: 1.9417 - acc: 0.2391 - val_loss: 1.9101 - val_acc: 0.2455
Epoch 7/100
1174/1174 [==============================] - 735s 626ms/step - loss: 1.9400 - acc: 0.2384 - val_loss: 1.9101 - val_acc: 0.2528
Epoch 8/100
1174/1174 [======

KeyboardInterrupt: 

In [ ]:
acc = history_VGG_aug.history['acc']
val_acc = history_VGG_aug.history['val_acc']
loss = history_VGG_aug.history['loss']
val_loss = history_VGG_aug.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs, acc, 'bo', label="Training acc")
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label="Training loss")
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.show()

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_VGG_aug.history) 

# # save to json:  
# hist_json_file = 'history_RESNET_before.json' 
# with open(hist_json_file, mode='w') as f:
#     hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history_VGG_aug_before_transfer.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    
#pd.read_csv('history_RESNET.csv')

In [ ]:
model_VGG_aug.trainable=True
set_trainable=False
for layer in model_VGG_aug.layers:
    if layer.name =='block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
earlystopping = callbacks.EarlyStopping(monitor ="val_loss",
                                        mode ="min", 
                                        patience = 20)

In [ ]:
num_epoch = 200
history_vgg_aug_after_transfer = model_VGG_aug.fit(train_generator_aug,
                    epochs=num_epoch,
                    verbose=1,
                    validation_data=valid_generator_aug,
                   callbacks =[earlystopping])

In [ ]:
acc = history_vgg_aug_after_transfer.history['acc']
val_acc = history_vgg_aug_after_transfer.history['val_acc']
loss = history_vgg_aug_after_transfer.history['loss']
val_loss = history_vgg_aug_after_transfer.history['val_loss']

epochs = range(1,len(acc)+1)

plt.plot(epochs, acc, 'bo', label="Training acc")
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label="Training loss")
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.show()

In [ ]:
# convert the history.history dict to a pandas DataFrame:     
hist_df = pd.DataFrame(history_vgg_aug_after_transfer.history) 

# # save to json:  
# hist_json_file = 'history_RESNET_before.json' 
# with open(hist_json_file, mode='w') as f:
#     hist_df.to_json(f)

# or save to csv: 
hist_csv_file = 'history_vgg_aug_after_transfer.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)
    
#pd.read_csv('history_RESNET.csv')

In [ ]:
#save the model
model_VGG_aug.save('results/VGG_with_aug')

In [ ]:
#load the model
model_VGG_aug_loaded = tf.keras.models.load_model('results/VGG_with_aug')
# Check its architecture
model_seq_aug_loaded.summary()